#### importing modules & reproducibility

In [26]:
# import modules
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

# Fixing random state for reproducibility
np.random.seed(387)

# ***Preprocessing***

In [27]:
# read data
characters = pd.read_csv('datasets/Characters.csv', sep=';')
hp1 = pd.read_csv('datasets/Harry Potter 1.csv', sep=';')
hp2 = pd.read_csv('datasets/Harry Potter 2.csv', sep=';')
hp3 = pd.read_csv('datasets/Harry Potter 3.csv', sep=';')

## ***profiling structure***
### fixing books data

In [28]:
trilogy = [hp1, hp2, hp3] 

# changing books columns and values to lower case
'''if not other cols would be created because of name differences'''
for book in trilogy:
    book.columns = hp1.columns.str.lower()
    book['character'] = book['character'].str.lower() # character column
    book['sentence'] = book['sentence'].str.lower()   # sentence column

In [29]:
# vertical concatenation of books
combined = pd.concat([hp1, hp2, hp3], ignore_index=True)
combined.head()

,character,sentence
0,dumbledore,"i should've known that you would be here, prof..."
1,mcgonagall,"good evening, professor dumbledore."
2,mcgonagall,"are the rumors true, albus?"
3,dumbledore,"i'm afraid so, professor."
4,dumbledore,the good and the bad.


### fixing characters data

In [30]:
# changing books columns and values to lower case
'''if not other cols would be created because of name differences'''
characters.columns = characters.columns.str.lower()

In [47]:
# columns to date format
characters['birth'] = pd.to_datetime(characters['birth'], errors='coerce')
characters['death'] = pd.to_datetime(characters['death'], errors='coerce')

In [50]:
# get non numerical values
non_numerical = characters.select_dtypes(exclude=['int64','float64','datetime','timedelta'])

In [52]:
# transform to dataframe
characters = pd.DataFrame(characters)
characters

TypeError: string operation on non-string array

#### inspecting tables

In [11]:
# column names
print(characters.columns,'\n')
print(hp1.columns)

Index(['id', 'name', 'gender', 'job', 'house', 'wand', 'patronus', 'species',
       'blood status', 'hair colour', 'eye colour', 'loyalty', 'skills',
       'birth', 'death'],
      dtype='object') 

Index(['character', 'sentence'], dtype='object')


# Plots :
### Characters with the most sentences
#### top 15 of the first 3 movies 